

# NWB File Basics

This example will focus on the basics of working with an :py:class:`~pynwb.file.NWBFile` object,
including writing and reading of an NWB file, and giving you an introduction to the basic data types.
Before we dive into code showing how to use an :py:class:`~pynwb.file.NWBFile`, we first provide
a brief overview of the basic concepts of NWB.


## Background: Basic concepts

In the [NWB Format](https://nwb-schema.readthedocs.io), each experiment session is typically
represented by a separate NWB file. NWB files are represented in PyNWB by :py:class:`~pynwb.file.NWBFile`
objects which provide functionality for creating and retrieving:

 * `timeseries_overview` datasets -- objects for storing time series data
 * `modules_overview` -- objects for storing and grouping analyses, and
 * experiment metadata and other metadata related to data provenance.

The following sections describe the :py:class:`~pynwb.base.TimeSeries` and :py:class:`~pynwb.base.ProcessingModule`
classes in further detail.


### TimeSeries

:py:class:`~pynwb.base.TimeSeries` objects store time series data and correspond to the *TimeSeries* specifications
provided by the `NWB Format`_. Like the NWB specification, :py:class:`~pynwb.base.TimeSeries` Python objects
follow an object-oriented inheritance pattern, i.e., the class :py:class:`~pynwb.base.TimeSeries`
serves as the base class for all other :py:class:`~pynwb.base.TimeSeries` types, such as,
:py:class:`~pynwb.ecephys.ElectricalSeries`, which itself may have further subtypes, e.g.,
:py:class:`~pynwb.ecephys.SpikeEventSeries`.

.. seealso::

    For your reference, NWB defines the following main :py:class:`~pynwb.base.TimeSeries` subtypes:

    * **Extracellular electrophysiology:**
      :py:class:`~pynwb.ecephys.ElectricalSeries`, :py:class:`~pynwb.ecephys.SpikeEventSeries`

    * **Intracellular electrophysiology:**
      :py:class:`~pynwb.icephys.PatchClampSeries` is the base type for all intracellular time series, which
      is further refined into subtypes depending on the type of recording:
      :py:class:`~pynwb.icephys.CurrentClampSeries`,
      :py:class:`~pynwb.icephys.IZeroClampSeries`,
      :py:class:`~pynwb.icephys.CurrentClampStimulusSeries`,
      :py:class:`~pynwb.icephys.VoltageClampSeries`,
      :py:class:`~pynwb.icephys.VoltageClampStimulusSeries`.

    * **Optical physiology and imaging:** :py:class:`~pynwb.image.ImageSeries` is the base type
      for image recordings and is further refined by the
      :py:class:`~pynwb.image.ImageMaskSeries`,
      :py:class:`~pynwb.image.OpticalSeries`,
      :py:class:`~pynwb.ophys.OnePhotonSeries`, and
      :py:class:`~pynwb.ophys.TwoPhotonSeries` types.
      Other related time series types are:
      :py:class:`~pynwb.image.IndexSeries`,
      :py:class:`~pynwb.ophys.RoiResponseSeries`.

    * **Others:** :py:class:`~pynwb.ogen.OptogeneticSeries`,
      :py:class:`~pynwb.behavior.SpatialSeries`,
      :py:class:`~pynwb.misc.DecompositionSeries`,
      :py:class:`~pynwb.misc.AnnotationSeries`,
      :py:class:`~pynwb.misc.AbstractFeatureSeries`,
      :py:class:`~pynwb.misc.IntervalSeries`.



### Processing Modules

Processing modules are objects that group together common analyses done during processing of data. They
often hold data of different processing/analysis data types.

.. seealso::

    For your reference, NWB defines the following main processing/analysis data types:

    * **Behavior:** :py:class:`~pynwb.behavior.BehavioralEpochs`,
      :py:class:`~pynwb.behavior.BehavioralEvents`,
      :py:class:`~pynwb.behavior.BehavioralTimeSeries`,
      :py:class:`~pynwb.behavior.CompassDirection`,
      :py:class:`~pynwb.behavior.PupilTracking`,
      :py:class:`~pynwb.behavior.Position`,
      :py:class:`~pynwb.behavior.EyeTracking`.

    * **Extracellular electrophysiology:** :py:class:`~pynwb.ecephys.EventDetection`,
      :py:class:`~pynwb.ecephys.EventWaveform`,
      :py:class:`~pynwb.ecephys.FeatureExtraction`,
      :py:class:`~pynwb.ecephys.FilteredEphys`,
      :py:class:`~pynwb.ecephys.LFP`.

    * **Optical physiology:** :py:class:`~pynwb.ophys.DfOverF`,
      :py:class:`~pynwb.ophys.Fluorescence`,
      :py:class:`~pynwb.ophys.ImageSegmentation`,
      :py:class:`~pynwb.ophys.MotionCorrection`.

    * **Others:** :py:class:`~pynwb.base.Images`.

    * **TimeSeries:** Any :py:class:`~pynwb.base.TimeSeries` can be used to store processing/analysis data.

NWB organizes data into different groups depending on the type of data. Groups can be thought of
as folders within the file. Here are some of the groups within an :py:class:`~pynwb.file.NWBFile` and the types of
data they are intended to store:

* **acquisition**: raw, acquired data that should never change
* **processing**: processed data, typically the results of preprocessing algorithms and could change
* **analysis**: results of data analysis
* **stimuli**: stimuli used in the experiment (e.g., images, videos, light pulses)

The following examples will reference variables that may not be defined within the block they are used in. For
clarity, we define them here:


In [1]:
from datetime import datetime
from uuid import uuid4

import numpy as np
from dateutil import tz

from pynwb import NWBHDF5IO, NWBFile, TimeSeries
from pynwb.behavior import Position, SpatialSeries
from pynwb.file import Subject
from pynwb.misc import AnnotationSeries


## The NWB file

An :py:class:`~pynwb.file.NWBFile` represents a single session of an experiment.
Each :py:class:`~pynwb.file.NWBFile` must have a session description, identifier, and session start time.
Importantly, the session start time is the reference time for all timestamps in the file.
For instance, an event with a timestamp of 0 in the file means the event
occurred exactly at the session start time.

Create an :py:class:`~pynwb.file.NWBFile` object with the required fields
(:py:attr:`~pynwb.file.NWBFile.session_description`, :py:attr:`~pynwb.file.NWBFile.identifier`,
:py:attr:`~pynwb.file.NWBFile.session_start_time`) and additional metadata.

<div class="alert alert-info"><h4>Note</h4><p>Use keyword arguments when constructing :py:class:`~pynwb.file.NWBFile` objects.</p></div>




In [2]:
session_start_time = datetime(2018, 4, 25, 2, 30, 3, tzinfo=tz.gettz("US/Pacific"))

nwbfile = NWBFile(
    session_description="Mouse exploring an open field",  # required
    identifier=str(uuid4()),  # required
    session_start_time=session_start_time,  # required
    session_id="session_1234",  # optional
    experimenter=[
        "Baggins, Bilbo",
    ],  # optional
    lab="Bag End Laboratory",  # optional
    institution="University of Middle Earth at the Shire",  # optional
    experiment_description="I went on an adventure to reclaim vast treasures.",  # optional
    keywords=["behavior", "exploration", "wanderlust"],  # optional
    related_publications="doi:10.1016/j.neuron.2016.12.011",  # optional
)
nwbfile

root pynwb.file.NWBFile at 0x139930438585408
Fields:
  experiment_description: I went on an adventure to reclaim vast treasures.
  experimenter: ['Baggins, Bilbo']
  file_create_date: [datetime.datetime(2024, 11, 29, 16, 47, 46, 825874, tzinfo=tzlocal())]
  identifier: 03cf995f-1f6c-4bc1-9698-42323e74a16d
  institution: University of Middle Earth at the Shire
  keywords: ['behavior' 'exploration' 'wanderlust']
  lab: Bag End Laboratory
  related_publications: ['doi:10.1016/j.neuron.2016.12.011']
  session_description: Mouse exploring an open field
  session_id: session_1234
  session_start_time: 2018-04-25 02:30:03-07:00
  timestamps_reference_time: 2018-04-25 02:30:03-07:00

<div class="alert alert-info"><h4>Note</h4><p>See the [NWBFile Best Practices](https://nwbinspector.readthedocs.io/en/dev/best_practices/nwbfile_metadata.html#file-metadata)
    for detailed information about the arguments to
    :py:class:`~pynwb.file.NWBFile`</p></div>




## Subject Information

In the :py:class:`~pynwb.file.Subject` object we can store information about the experiment subject,
such as ``age``, ``species``, ``genotype``, ``sex``, and a ``description``.

.. only:: html

  .. image:: ../../_static/Subject.svg
    :width: 150
    :alt: subject UML diagram
    :align: center

.. only:: latex

  .. image:: ../../_static/Subject.png
    :width: 150
    :alt: subject UML diagram
    :align: center

The fields in the :py:class:`~pynwb.file.Subject` object are all free-form text (any format will be valid),
however it is recommended to follow particular conventions to help software tools interpret the data:

* **age**: [ISO 8601 Duration format](https://en.wikipedia.org/wiki/ISO_8601#Durations), e.g., ``"P90D"`` for 90
  days old
* **species**: The formal Latin binomial nomenclature, e.g., ``"Mus musculus"``, ``"Homo sapiens"``
* **sex**: Single letter abbreviation, e.g., ``"F"`` (female), ``"M"`` (male), ``"U"`` (unknown), and ``"O"`` (other)

Add the subject information to the :py:class:`~pynwb.file.NWBFile`
by setting the ``subject`` field to a new :py:class:`~pynwb.file.Subject` object.



In [3]:
subject = Subject(
    subject_id="001",
    age="P90D",
    description="mouse 5",
    species="Mus musculus",
    sex="M",
)

nwbfile.subject = subject
subject

subject pynwb.file.Subject at 0x139929307744336
Fields:
  age: P90D
  age__reference: birth
  description: mouse 5
  sex: M
  species: Mus musculus
  subject_id: 001


## Time Series Data

:py:class:`~pynwb.base.TimeSeries` is a common base class for measurements sampled over time,
and provides fields for ``data`` and ``timestamps`` (regularly or irregularly sampled).
You will also need to supply the ``name`` and ``unit`` of measurement
([SI unit](https://en.wikipedia.org/wiki/International_System_of_Units)).

<img src="file://../../_static/TimeSeries.png" width="200" alt="timeseries UML diagram" align="center">

For instance, we can store a :py:class:`~pynwb.base.TimeSeries` data where recording started
``0.0`` seconds after ``start_time`` and sampled every second (1 Hz):



In [4]:
data = np.arange(100, 200, 10)
time_series_with_rate = TimeSeries(
    name="test_timeseries",
    description="an example time series",
    data=data,
    unit="m",
    starting_time=0.0,
    rate=1.0,
)
time_series_with_rate

test_timeseries pynwb.base.TimeSeries at 0x139929307696864
Fields:
  comments: no comments
  conversion: 1.0
  data: [100 110 120 130 140 150 160 170 180 190]
  description: an example time series
  offset: 0.0
  rate: 1.0
  resolution: -1.0
  starting_time: 0.0
  starting_time_unit: seconds
  unit: m

For irregularly sampled recordings, we need to provide the ``timestamps`` for the ``data``:



In [5]:
timestamps = np.arange(10.)
time_series_with_timestamps = TimeSeries(
    name="test_timeseries",
    description="an example time series",
    data=data,
    unit="m",
    timestamps=timestamps,
)
time_series_with_timestamps

test_timeseries pynwb.base.TimeSeries at 0x139930438202896
Fields:
  comments: no comments
  conversion: 1.0
  data: [100 110 120 130 140 150 160 170 180 190]
  description: an example time series
  interval: 1
  offset: 0.0
  resolution: -1.0
  timestamps: [0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]
  timestamps_unit: seconds
  unit: m

:py:class:`~pynwb.base.TimeSeries` objects can be added directly to :py:class:`~pynwb.file.NWBFile` using:

* :py:meth:`.NWBFile.add_acquisition` to add *acquisition* data (raw, acquired data that should never change),
* :py:meth:`.NWBFile.add_stimulus` to add *stimulus* data, or
* :py:meth:`.NWBFile.add_stimulus_template` to store *stimulus templates*.




In [6]:
nwbfile.add_acquisition(time_series_with_timestamps)

We can access the :py:class:`~pynwb.base.TimeSeries` object ``'test_timeseries'``
in :py:class:`~pynwb.file.NWBFile` from ``acquisition``:



In [7]:
nwbfile.acquisition["test_timeseries"]

test_timeseries pynwb.base.TimeSeries at 0x139930438202896
Fields:
  comments: no comments
  conversion: 1.0
  data: [100 110 120 130 140 150 160 170 180 190]
  description: an example time series
  interval: 1
  offset: 0.0
  resolution: -1.0
  timestamps: [0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]
  timestamps_unit: seconds
  unit: m

or using the method :py:meth:`.NWBFile.get_acquisition`:



In [8]:
nwbfile.get_acquisition("test_timeseries")

test_timeseries pynwb.base.TimeSeries at 0x139930438202896
Fields:
  comments: no comments
  conversion: 1.0
  data: [100 110 120 130 140 150 160 170 180 190]
  description: an example time series
  interval: 1
  offset: 0.0
  resolution: -1.0
  timestamps: [0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]
  timestamps_unit: seconds
  unit: m

### Other Types of Time Series

As mentioned previously, there are many subtypes of :py:class:`~pynwb.base.TimeSeries` that are used to store
different kinds of data. One example is :py:class:`~pynwb.misc.AnnotationSeries`, a subclass of 
:py:class:`~pynwb.base.TimeSeries` that stores text-based records about the experiment. Similarly to our
:py:class:`~pynwb.base.TimeSeries` example above, we can create an :py:class:`~pynwb.misc.AnnotationSeries` 
object with text information about a stimulus and add it to the stimulus group in 
the :py:class:`~pynwb.file.NWBFile`. 



In [9]:
annotations = AnnotationSeries(
    name='airpuffs',
    data=['Left Airpuff', 'Right Airpuff', 'Right Airpuff'],
    description='Airpuff events delivered to the animal',
    timestamps=[1.0, 3.0, 8.0],
)

nwbfile.add_stimulus(annotations)

This approach of creating a :py:class:`~pynwb.base.TimeSeries` object and adding it to the appropriate 
:py:class:`~pynwb.file.NWBFile` group can be used for all subtypes of :py:class:`~pynwb.base.TimeSeries` data.




## Spatial Series and Position

:py:class:`~pynwb.behavior.SpatialSeries` is a subclass of :py:class:`~pynwb.base.TimeSeries`
that represents the spatial position of an animal over time.

.. only:: html

  .. image:: ../../_static/SpatialSeries.svg
    :width: 200
    :alt: spatialseries UML diagram
    :align: center

.. only:: latex

  .. image:: ../../_static/SpatialSeries.png
    :width: 200
    :alt: spatialseries UML diagram
    :align: center

Create a :py:class:`~pynwb.behavior.SpatialSeries` object named ``"SpatialSeries"`` with some fake data.



In [10]:
# create fake data with shape (50, 2)
# the first dimension should always represent time
position_data = np.array([np.linspace(0, 10, 50), np.linspace(0, 8, 50)]).T
position_timestamps = np.linspace(0, 50).astype(float) / 200

spatial_series_obj = SpatialSeries(
    name="SpatialSeries",
    description="(x,y) position in open field",
    data=position_data,
    timestamps=position_timestamps,
    reference_frame="(0,0) is bottom left corner",
)
spatial_series_obj

SpatialSeries pynwb.behavior.SpatialSeries at 0x139929307696192
Fields:
  comments: no comments
  conversion: 1.0
  data: [[ 0.          0.        ]
 [ 0.20408163  0.16326531]
 [ 0.40816327  0.32653061]
 [ 0.6122449   0.48979592]
 [ 0.81632653  0.65306122]
 [ 1.02040816  0.81632653]
 [ 1.2244898   0.97959184]
 [ 1.42857143  1.14285714]
 [ 1.63265306  1.30612245]
 [ 1.83673469  1.46938776]
 [ 2.04081633  1.63265306]
 [ 2.24489796  1.79591837]
 [ 2.44897959  1.95918367]
 [ 2.65306122  2.12244898]
 [ 2.85714286  2.28571429]
 [ 3.06122449  2.44897959]
 [ 3.26530612  2.6122449 ]
 [ 3.46938776  2.7755102 ]
 [ 3.67346939  2.93877551]
 [ 3.87755102  3.10204082]
 [ 4.08163265  3.26530612]
 [ 4.28571429  3.42857143]
 [ 4.48979592  3.59183673]
 [ 4.69387755  3.75510204]
 [ 4.89795918  3.91836735]
 [ 5.10204082  4.08163265]
 [ 5.30612245  4.24489796]
 [ 5.51020408  4.40816327]
 [ 5.71428571  4.57142857]
 [ 5.91836735  4.73469388]
 [ 6.12244898  4.89795918]
 [ 6.32653061  5.06122449]
 [ 6.53061224  5.2244898 ]
 [ 6.73469388  5.3877551 ]
 [ 6.93877551  5.55102041]
 [ 7.14285714  5.71428571]
 [ 7.34693878  5.87755102]
 [ 7.55102041  6.04081633]
 [ 7.75510204  6.20408163]
 [ 7.95918367  6.36734694]
 [ 8.16326531  6.53061224]
 [ 8.36734694  6.69387755]
 [ 8.57142857  6.85714286]
 [ 8.7755102   7.02040816]
 [ 8.97959184  7.18367347]
 [ 9.18367347  7.34693878]
 [ 9.3877551   7.51020408]
 [ 9.59183673  7.67346939]
 [ 9.79591837  7.83673469]
 [10.          8.        ]]
  description: (x,y) position in open field
  interval: 1
  offset: 0.0
  reference_frame: (0,0) is bottom left corner
  resolution: -1.0
  timestamps: [0.         0.00510204 0.01020408 0.01530612 0.02040816 0.0255102
 0.03061224 0.03571429 0.04081633 0.04591837 0.05102041 0.05612245
 0.06122449 0.06632653 0.07142857 0.07653061 0.08163265 0.08673469
 0.09183673 0.09693878 0.10204082 0.10714286 0.1122449  0.11734694
 0.12244898 0.12755102 0.13265306 0.1377551  0.14285714 0.14795918
 0.15306122 0.15816327 0.16326531 0.16836735 0.17346939 0.17857143
 0.18367347 0.18877551 0.19387755 0.19897959 0.20408163 0.20918367
 0.21428571 0.21938776 0.2244898  0.22959184 0.23469388 0.23979592
 0.24489796 0.25      ]
  timestamps_unit: seconds
  unit: meters

To help data analysis and visualization tools know that this :py:class:`~pynwb.behavior.SpatialSeries` object
represents the position of the subject, store the :py:class:`~pynwb.behavior.SpatialSeries` object inside
of a :py:class:`~pynwb.behavior.Position` object, which can hold one or more :py:class:`~pynwb.behavior.SpatialSeries`
objects.

.. only:: html

  .. image:: ../../_static/Position.svg
    :width: 450
    :alt: position UML diagram
    :align: center

.. only:: latex

  .. image:: ../../_static/Position.png
    :width: 450
    :alt: position UML diagram
    :align: center

Create a :py:class:`~pynwb.behavior.Position` object named ``"Position"`` [#]_.



In [11]:
# name is set to "Position" by default
position_obj = Position(spatial_series=spatial_series_obj)
position_obj

Position pynwb.behavior.Position at 0x139929307745776
Fields:
  spatial_series: {
    SpatialSeries <class 'pynwb.behavior.SpatialSeries'>
  }

## Behavior Processing Module

:py:class:`~pynwb.base.ProcessingModule` is a container for data interfaces that are related to a particular
processing workflow. NWB differentiates between raw, acquired data (*acquisition*), which should never change,
and processed data (*processing*), which are the results of preprocessing algorithms and could change.
Processing modules can be thought of as folders within the file for storing the related processed data.

.. tip:: Use the NWB schema module names as processing module names where appropriate.
   These are: ``"behavior"``, ``"ecephys"``, ``"icephys"``, ``"ophys"``, ``"ogen"``, and ``"misc"``.

Let's assume that the subject's position was computed from a video tracking algorithm,
so it would be classified as processed data.

Create a processing module called ``"behavior"`` for storing behavioral data in the :py:class:`~pynwb.file.NWBFile`
and add the :py:class:`~pynwb.behavior.Position` object to the processing module using the method
:py:meth:`.NWBFile.create_processing_module`:



In [12]:
behavior_module = nwbfile.create_processing_module(
    name="behavior", description="processed behavioral data"
)
behavior_module.add(position_obj)
behavior_module

behavior pynwb.base.ProcessingModule at 0x139929307490048
Fields:
  data_interfaces: {
    Position <class 'pynwb.behavior.Position'>
  }
  description: processed behavioral data

.. only:: html

  .. image:: ../../_static/Behavior.svg
    :width: 600
    :alt: behavior UML diagram
    :align: center

.. only:: latex

  .. image:: ../../_static/Behavior.png
    :width: 600
    :alt: behavior UML diagram
    :align: center

Once the behavior processing module is added to the :py:class:`~pynwb.file.NWBFile`,
you can access it with:



In [13]:
nwbfile.processing["behavior"]

behavior pynwb.base.ProcessingModule at 0x139929307490048
Fields:
  data_interfaces: {
    Position <class 'pynwb.behavior.Position'>
  }
  description: processed behavioral data

## Time Intervals


The following provides a brief introduction to managing annotations in time via
:py:class:`~pynwb.epoch.TimeIntervals`. See the `time_intervals` tutorial
for a more detailed introduction to :py:class:`~pynwb.epoch.TimeIntervals`.

### Trials

Trials are stored in :py:class:`~pynwb.epoch.TimeIntervals`, which is
a subclass of :py:class:`~hdmf.common.table.DynamicTable`.
:py:class:`~hdmf.common.table.DynamicTable` is used to store
tabular metadata throughout NWB, including trials, electrodes and sorted units. This
class offers flexibility for tabular data by allowing required columns, optional
columns, and custom columns which are not defined in the standard.

.. only:: html

  .. image:: ../../_static/Trials.svg
    :width: 300
    :alt: trials UML diagram
    :align: center

.. only:: latex

  .. image:: ../../_static/Trials.png
    :width: 300
    :alt: trials UML diagram
    :align: center

The ``trials`` :py:class:`~pynwb.epoch.TimeIntervals` class can be thought of
as a table with this structure:

<img src="file://../../_static/trials_example.png" width="400" alt="trials table example" align="center">

By default, :py:class:`~pynwb.epoch.TimeIntervals` objects only require ``start_time``
and ``stop_time`` of each trial. Additional columns can be added using
the method :py:meth:`.NWBFile.add_trial_column`. When all the desired custom columns
have been defined, use the :py:meth:`.NWBFile.add_trial` method to add each row.
In this case, we will add one custom column to the trials table named "correct"
which will take a boolean array, then add two trials as rows of the table.



In [14]:
nwbfile.add_trial_column(
    name="correct",
    description="whether the trial was correct",
)
nwbfile.add_trial(start_time=1.0, stop_time=5.0, correct=True)
nwbfile.add_trial(start_time=6.0, stop_time=10.0, correct=False)

:py:class:`~hdmf.common.table.DynamicTable` and its subclasses can be converted to a pandas
:py:class:`~pandas.DataFrame` for convenient analysis using :py:meth:`~hdmf.common.table.DynamicTable.to_dataframe`.



In [15]:
nwbfile.trials.to_dataframe()

,start_time,stop_time,correct
id,,,
0,1.0,5.0,True
1,6.0,10.0,False



## Writing an NWB file

Writing of an NWB file is carried out using the :py:class:`~pynwb.NWBHDF5IO` class [#]_.

To write an :py:class:`~pynwb.file.NWBFile`, use the :py:meth:`~hdmf.backends.io.HDMFIO.write` method.



In [16]:
io = NWBHDF5IO("basics_tutorial.nwb", mode="w")
io.write(nwbfile)
io.close()

You can also use :py:meth:`~pynwb.NWBHDF5IO` as a context manager:



In [17]:
with NWBHDF5IO("basics_tutorial.nwb", "w") as io:
    io.write(nwbfile)


## Reading an NWB file

As with writing, reading is also carried out using the :py:class:`~pynwb.NWBHDF5IO` class.
To read the NWB file we just wrote, create another :py:class:`~pynwb.NWBHDF5IO` object with the mode set to ``"r"``,
and use the :py:meth:`~pynwb.NWBHDF5IO.read` method to retrieve an
:py:class:`~pynwb.file.NWBFile` object.

Data arrays are read passively from the file.
Accessing the ``data`` attribute of the :py:class:`~pynwb.base.TimeSeries` object
does not read the data values, but presents an HDF5 object that can be indexed to read data.
You can use the ``[:]`` operator to read the entire data array into memory.



In [18]:
with NWBHDF5IO("basics_tutorial.nwb", "r") as io:
    read_nwbfile = io.read()
    print(read_nwbfile.acquisition["test_timeseries"])
    print(read_nwbfile.acquisition["test_timeseries"].data[:])

test_timeseries pynwb.base.TimeSeries at 0x139929305892416
Fields:
  comments: no comments
  conversion: 1.0
  data: <HDF5 dataset "data": shape (10,), type "<i8">
  description: an example time series
  interval: 1
  offset: 0.0
  resolution: -1.0
  timestamps: <HDF5 dataset "timestamps": shape (10,), type "<f8">
  timestamps_unit: seconds
  unit: m

[100 110 120 130 140 150 160 170 180 190]


It is often preferable to read only a portion of the data.
To do this, index or slice into the ``data`` attribute just like you
index or slice a numpy array.



In [19]:
with NWBHDF5IO("basics_tutorial.nwb", "r") as io:
    read_nwbfile = io.read()
    print(read_nwbfile.acquisition["test_timeseries"].data[:2])

[100 110]


<div class="alert alert-info"><h4>Note</h4><p>If you use :py:class:`~pynwb.NWBHDF5IO` as a context manager during read,
    be aware that the :py:class:`~pynwb.NWBHDF5IO` gets closed and when the
    context completes and the data will not be available outside of the
    context manager [#]_.</p></div>



### Accessing data

We can also access the :py:class:`~pynwb.behavior.SpatialSeries` data by referencing the names
of the objects in the hierarchy that contain it. We can access a processing module by indexing
``nwbfile.processing`` with the name of the processing module, ``"behavior"``.

Then, we can access the :py:class:`~pynwb.behavior.Position` object inside of the ``"behavior"``
processing module by indexing it with the name of the :py:class:`~pynwb.behavior.Position` object,
``"Position"``.

Finally, we can access the :py:class:`~pynwb.behavior.SpatialSeries` object inside of the
:py:class:`~pynwb.behavior.Position` object by indexing it with the name of the
:py:class:`~pynwb.behavior.SpatialSeries` object, ``"SpatialSeries"``.



In [20]:
with NWBHDF5IO("basics_tutorial.nwb", "r") as io:
    read_nwbfile = io.read()
    print(read_nwbfile.processing["behavior"])
    print(read_nwbfile.processing["behavior"]["Position"])
    print(read_nwbfile.processing["behavior"]["Position"]["SpatialSeries"])

behavior pynwb.base.ProcessingModule at 0x139929306882736
Fields:
  data_interfaces: {
    Position <class 'pynwb.behavior.Position'>
  }
  description: processed behavioral data

Position pynwb.behavior.Position at 0x139929306883024
Fields:
  spatial_series: {
    SpatialSeries <class 'pynwb.behavior.SpatialSeries'>
  }

SpatialSeries pynwb.behavior.SpatialSeries at 0x139929306883168
Fields:
  comments: no comments
  conversion: 1.0
  data: <HDF5 dataset "data": shape (50, 2), type "<f8">
  description: (x,y) position in open field
  interval: 1
  offset: 0.0
  reference_frame: (0,0) is bottom left corner
  resolution: -1.0
  timestamps: <HDF5 dataset "timestamps": shape (50,), type "<f8">
  timestamps_unit: seconds
  unit: meters




## Appending to an NWB file

To append to a file, read it with :py:class:`~pynwb.NWBHDF5IO` and set the ``mode`` argument to ``'a'``.
After you have read the file, you can add [#]_ new data to it using the standard write/add functionality demonstrated
above. Let's see how this works by adding another :py:class:`~pynwb.base.TimeSeries` to acquisition.



In [21]:
io = NWBHDF5IO("basics_tutorial.nwb", mode="a")
nwbfile = io.read()

data = np.arange(100, 200, 10)
timestamps = np.arange(10.)
new_time_series = TimeSeries(
    name="new_time_series",
    description="a new time series",
    data=data,
    timestamps=timestamps,
    unit="n.a.",
)
nwbfile.add_acquisition(new_time_series)

Finally, write the changes back to the file and close it.



In [22]:
io.write(nwbfile)
io.close()

.. [#] Some data interface objects have a default name. This default name is the type of the data interface. For
   example, the default name for :py:class:`~pynwb.ophys.ImageSegmentation` is "ImageSegmentation" and the default
   name for :py:class:`~pynwb.ecephys.EventWaveform` is "EventWaveform".

.. [#] HDF5 is the primary backend supported by NWB.

.. [#] Neurodata sets can be *very* large, so individual components of the dataset are only loaded into memory when
   you request them. This functionality is only possible if an open file handle is kept around until users want to
   load data.

.. [#] NWB only supports *adding* to files. Removal and modifying of existing data is not allowed.

